# 1. IMPORTING AND READING CSVS

In [44]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# movies.csv
url = 'https://drive.google.com/file/d/1EknR8ceW4HEkjW-41F4z_IVku0RZERWH/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
movies = pd.read_csv(path)

# links.csv
url = 'https://drive.google.com/file/d/1F0jEbRxpqt1L6B_OjiLK29wZtj5JIocV/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
links = pd.read_csv(path)

# 'ratings.csv'
url = 'https://drive.google.com/file/d/1IdcUbfqlIIY0djoAtTKEDXhL6TCXyCoG/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
ratings = pd.read_csv(path) 

# tags.csv
url = 'https://drive.google.com/file/d/1kOIkbYH6xjM_38EKqienvHtzmWUGunt4/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
tags = pd.read_csv(path)

# CLEANING

In [45]:
ratings.drop(columns = "timestamp")

Ratings = ratings.rename({'userId ': 'userId', 'movieId ': 'movieId', 'rating ':'rating'}, axis=1) 
Movies = movies.rename({'movieId ': 'movieId', 'title                                                                                                                                                          ':'title'
, 'genres' :'genres'}, axis=1) 

Movies['title'] = Movies['title'].str.strip()

# MERGING

In [46]:
Ratings_Movies = Ratings.merge(Movies, on='movieId')

# pivot_table FOR RECOMMENDERS


In [47]:
users_items = pd.pivot_table(data=Ratings_Movies, values='rating', index='userId', columns='movieId')
users_items.fillna(0, inplace=True)

# USER BASED RECOMMENDER 

In [48]:

# CALCULATING COSINE SIMILARITY
user_similarities = pd.DataFrame(cosine_similarity(users_items),
                                 columns=users_items.index, 
                                 index=users_items.index)

# compute the weights for one user
user_id = 1

weights = (
    user_similarities.query("userId!=@user_id")[user_id] / sum(user_similarities.query("userId!=@user_id")[user_id])
          )

# TRY AND ERROR

In [49]:
ratings.drop(columns = "timestamp")

Ratings = ratings.rename({'userId ': 'userId', 'movieId ': 'movieId', 'rating ':'rating'}, axis=1) 
Movies = movies.rename({'movieId ': 'movieId', 'title                                                                                                                                                          ':'title'
, 'genres' :'genres'}, axis=1) 

Movies['title'] = Movies['title'].str.strip()

Ratings_Movies = Ratings.merge(Movies, on='movieId')

users_items = pd.pivot_table(data=Ratings_Movies, values='rating', index='userId', columns='movieId')
users_items.fillna(0, inplace=True)


# CALCULATING COSINE SIMILARITY
user_similarities = pd.DataFrame(cosine_similarity(users_items),
                              columns=users_items.index, 
                              index=users_items.index)

# compute the weights for one user
weights = (
user_similarities.query("userId!=@user_id")[user_id] / sum(user_similarities.query("userId!=@user_id")[user_id])
      )


# compute the weights for one user
weights = (
    user_similarities.query("userId!=@user_id")[user_id] / sum(user_similarities.query("userId!=@user_id")[user_id])
        )

# find items that have not been rated by user
users_items.loc[user_id,:]==0

# select restaurants that the inputed user has not visited
not_visited_restaurants = users_items.loc[users_items.index!=user_id, users_items.loc[user_id,:]==0]

# dot product between the not-visited-restaurants and the weights
weighted_averages = pd.DataFrame(not_visited_restaurants.T.dot(weights), columns=["predicted_rating"])

# find the top n recommendations
recommendations = weighted_averages.merge(Movies, left_index=True, right_on="movieId")
top_n = recommendations.sort_values("predicted_rating", ascending=False).head(5)
top_n["title"]


277                      Shawshank Redemption, The (1994)
507                     Terminator 2: Judgment Day (1991)
659                                 Godfather, The (1972)
2078                              Sixth Sense, The (1999)
3638    Lord of the Rings: The Fellowship of the Ring,...
Name: title, dtype: object

## method for user based

In [55]:
def recommended_for_user(userId, n):
    # compute the weights for one user
    user_similarities = pd.DataFrame(cosine_similarity(users_items),
                                 columns=users_items.index, 
                                 index=users_items.index)
    weights = (
        user_similarities.query("userId!=@user_id")[user_id] / sum(user_similarities.query("userId!=@user_id")[user_id])
            )

    # find items that have not been rated by user
    users_items.loc[user_id,:]==0

    # select restaurants that the inputed user has not visited
    not_visited_restaurants = users_items.loc[users_items.index!=user_id, users_items.loc[user_id,:]==0]

    # dot product between the not-visited-restaurants and the weights
    weighted_averages = pd.DataFrame(not_visited_restaurants.T.dot(weights), columns=["predicted_rating"])

    # find the top n recommendations
    recommendations = weighted_averages.merge(Movies, left_index=True, right_on="movieId")
    top_n = recommendations.sort_values("predicted_rating", ascending=False).head(n)
    return top_n

In [56]:
recommended_for_user(601, 5)

,predicted_rating,movieId,title,genres
277,2.654727,318,"Shawshank Redemption, The (1994)",Crime|Drama
507,2.087327,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi
659,1.859548,858,"Godfather, The (1972)",Crime|Drama
2078,1.663564,2762,"Sixth Sense, The (1999)",Drama|Horror|Mystery
3638,1.624820,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy


# ITEM BASED RECOMMENDER

In [17]:
Movies.columns


Index(['movieId', 'title', 'genres'], dtype='object')

In [33]:
def similar_movies_mital(choosen_movie,n):
  #choosen_movie = 1
  users_items = pd.pivot_table(data=ratings, values='rating ', index= 'userId ', columns='movieId ')

  movie_ratings = users_items.loc[:,choosen_movie]
  movie_ratings[movie_ratings>=0] # exclude NaNs

  similar_to_movie = users_items.corrwith(movie_ratings)
  corr_movies = pd.DataFrame(similar_to_movie, columns=['correlation_score'])
  corr_movies.dropna(inplace=True)
  rating = pd.DataFrame(ratings.groupby('movieId ')['rating '].mean())
  rating['rating_count'] = ratings.groupby('movieId ')['rating '].count()
  movie_corr_summary = corr_movies.join(rating['rating_count'])
  movie_corr_summary.drop(choosen_movie, inplace=True) # drop Tortas Locas itself

  Title =  Movies[['movieId', 'title', 'genres']]
  corr_summary_with_title =movie_corr_summary.merge(Title, left_index= True, right_index=True)

  corr_summary_with_title.set_index('title', inplace = True)

  top10 = corr_summary_with_title[corr_summary_with_title['rating_count']>=10].sort_values('correlation_score', ascending=False).head(6)
  
  return top10.index

In [34]:
similar_movies_mital(1,10)

Index(['Octagon, The (1980)', 'Emerald Forest, The (1985)', 'September (1987)',
       'Disgrace (2008)', 'Secret Window (2004)', 'Duck, You Sucker (1971)'],
      dtype='object', name='title')